In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

###  Carga de datos

In [2]:
auctions_18_20 = auctions_19_21 = auctions_20_22 = auctions_21_23 = auctions_24_26 = None
installs_18_20 = installs_19_21 = installs_20_22 = installs_21_23 = installs_24_26 = None
events_18_20 = events_19_21 = events_20_22 = events_21_23 = events_24_26 = None
clicks_18_20 = clicks_19_21 = clicks_20_22 = clicks_21_23 = clicks_24_26 = None

In [3]:
auctions_18_20 = pd.read_csv('windows/18_20/auctions.csv', parse_dates = ['date'], infer_datetime_format = True)
auctions_19_21 = pd.read_csv('windows/19_21/auctions.csv', parse_dates = ['date'], infer_datetime_format = True)
auctions_20_22 = pd.read_csv('windows/20_22/auctions.csv', parse_dates = ['date'], infer_datetime_format = True)
auctions_21_23 = pd.read_csv('windows/21_23/auctions.csv', parse_dates = ['date'], infer_datetime_format = True)
auctions_24_26 = pd.read_csv('windows/24_26/auctions.csv', parse_dates = ['date'], infer_datetime_format = True)

In [4]:
installs_18_20 = pd.read_csv('windows/18_20/installs.csv', parse_dates = ['created'], infer_datetime_format = True)
installs_19_21 = pd.read_csv('windows/19_21/installs.csv', parse_dates = ['created'], infer_datetime_format = True)
installs_20_22 = pd.read_csv('windows/20_22/installs.csv', parse_dates = ['created'], infer_datetime_format = True)
installs_21_23 = pd.read_csv('windows/21_23/installs.csv', parse_dates = ['created'], infer_datetime_format = True)
installs_24_26 = pd.read_csv('windows/24_26/installs.csv', parse_dates = ['created'], infer_datetime_format = True)

In [5]:
clicks_18_20 = pd.read_csv('windows/18_20/clicks.csv', parse_dates = ['created'], infer_datetime_format = True)
clicks_19_21 = pd.read_csv('windows/19_21/clicks.csv', parse_dates = ['created'], infer_datetime_format = True)
clicks_20_22 = pd.read_csv('windows/20_22/clicks.csv', parse_dates = ['created'], infer_datetime_format = True)
clicks_21_23 = pd.read_csv('windows/21_23/clicks.csv', parse_dates = ['created'], infer_datetime_format = True)
clicks_24_26 = pd.read_csv('windows/24_26/clicks.csv', parse_dates = ['created'], infer_datetime_format = True)

In [25]:
events_18_20 = pd.read_csv('windows/18_20/events.csv', parse_dates = ['date'], infer_datetime_format = True)
events_19_21 = pd.read_csv('windows/19_21/events.csv', parse_dates = ['date'], infer_datetime_format = True)
events_20_22 = pd.read_csv('windows/20_22/events.csv', parse_dates = ['date'], infer_datetime_format = True)
events_21_23 = pd.read_csv('windows/21_23/events.csv', parse_dates = ['date'], infer_datetime_format = True)
events_24_26 = pd.read_csv('windows/24_26/events.csv', parse_dates = ['date'], infer_datetime_format = True)

C:\Users\leobe\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_features_18_20 = pd.read_csv('windows/18_20/features.csv', low_memory=False)
df_features_18_20.set_index('ref_hash', inplace=True)

In [5]:
df_features_19_21 = pd.read_csv('windows/19_21/features.csv', low_memory=False)
df_features_19_21.set_index('ref_hash', inplace=True)

In [6]:
df_features_20_22 = pd.read_csv('windows/20_22/features.csv', low_memory=False)
df_features_20_22.set_index('ref_hash', inplace=True)

In [7]:
df_features_21_23 = pd.read_csv('windows/21_23/features.csv', low_memory=False)
df_features_21_23.set_index('ref_hash', inplace=True)

In [74]:
df_features_24_26 = pd.read_csv('windows/24_26/features.csv').drop_duplicates()  #check
df_features_24_26.set_index('ref_hash', inplace=True)

### Partición de los datos por ventanas

In [83]:
windows = [
    {
        "auctions": auctions_18_20,
        "installs": installs_18_20,
        "clicks": clicks_18_20,
        "events": events_18_20,
        "features": df_features_18_20,
        "name": "18_20"
    },
    {
        "auctions": auctions_19_21,
        "installs": installs_19_21,
        "clicks": clicks_19_21,
        "events": events_19_21,
        "features": df_features_19_21,        
        "name": "19_21"
    },
    {
        "auctions": auctions_20_22,
        "installs": installs_20_22,
        "clicks": clicks_20_22,
        "events": events_20_22,
        "features": df_features_20_22,        
        "name": "20_22"
    },
    {
        "auctions": auctions_21_23,
        "installs": installs_21_23,
        "clicks": clicks_21_23,
        "events": events_21_23,
        "features": df_features_21_23,        
        "name": "21_23"
    },
    {
        "auctions": auctions_24_26,
        "installs": installs_24_26,
        "clicks": clicks_24_26,
        "events": events_24_26,
        "features": df_features_24_26,
        "name": "24_26"
    }
]

### Funcion para agregar features

In [10]:
def add_feature(df_features, df_to_join, feature_name, fill = 0):
    if feature_name in df_features.columns.tolist():
        return df_features
    df_features = df_features.merge(df_to_join[[feature_name]], how = 'left', left_index = True, right_index = True).\
    fillna(fill)
    return df_features

In [11]:
def add_features_to_windows(windows, feature_function, feature_name):
    for window in windows:
        auctions = window["auctions"]
        installs = window["installs"]
        events = window["events"]
        clicks = window["clicks"]
        
        df_to_join = feature_function(auctions, installs, events, clicks)
        window["features"] = add_feature(window["features"], df_to_join, feature_name)
        
    save_changes(windows)
    return windows

In [12]:
def save_changes(windows):
    for window in windows:
        feature = window["features"]
        name = window["name"]
        feature.to_csv("windows/{}/features.csv".format(name), index = True)

 # Sección para añadir features

> *¡Atención! Antes de usar la función add_feature, setear como indice el id de los dispositivos en el df que contiene el feature que se desea agregar*

> ### Feature 1: ***cantidad de veces que el usuario aparece en una subasta***

In [47]:
def calculate_appearances_auctions(auctions, installs, events, clicks):
    auctions['appearances_in_auctions'] = 1
    auc_feature = auctions.groupby('ref_hash').agg({'appearances_in_auctions' : 'count'})
    return auc_feature

In [84]:
windows = add_features_to_windows(windows, calculate_appearances_auctions, 'appearances_in_auctions')

> ### Feature 2: ***¿apareció en una subasta el último día?***

In [16]:
def user_appeared_last_day(auctions, installs, events, clicks):
    ultimo = auctions["date"].max().day
    auc_feature2 = auctions.loc[auctions["date"].dt.day == ultimo]
    auc_feature2['user_appeared_last_day'] = 1
    auc_feature2 = auc_feature2.groupby('ref_hash').agg({'user_appeared_last_day' : 'first'})
    return auc_feature2

In [85]:
windows = add_features_to_windows(windows, user_appeared_last_day, "user_appeared_last_day")

> ### Feature 3: ***en promedio, ¿ cuánto tardo el usuario en reaparecer en una subasta?***

In [86]:
def calculate_time_to_reappear(auctions, installs, events, clicks):
    df_sorted = auctions.sort_values(["ref_hash", "date"], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted['ref_hash'] == \
          df_sorted['ref_hash'].shift(), df_sorted['date'] - df_sorted['date'].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].fillna(259200) #to check
    df_sorted = df_sorted.groupby('ref_hash').agg({'time_to_reappear': 'mean'})
    return df_sorted

In [87]:
windows = add_features_to_windows(windows, calculate_time_to_reappear, "time_to_reappear")

> ### Feature 4: ***Día de la semana, día del mes, hora del día, segundo de la hora, minuto de la hora***

In [12]:
def add_date_features(auctions, installs, events, clicks):
    # Día de la semana
    auctions["dayofweek"] = auctions.date.dt.dayofweek
    # Día del mes
    auctions["day"] = auctions.date.dt.day
    # Hora del día
    auctions["hour"] = auctions.date.dt.hour
    # Segundo de la hora
    auctions["second"] = auctions.date.dt.second
    # Minuto de la hora
    auctions["minute"] = auctions.date.dt.minute

    return auctions.set_index('ref_hash')

In [9]:
windows = add_features_to_windows(windows, add_date_features, "dayofweek")

NameError: name 'add_features_to_windows' is not defined

In [ ]:
windows = add_features_to_windows(windows, add_date_features, "day")

In [ ]:
windows = add_features_to_windows(windows, add_date_features, "second")

In [ ]:
windows = add_features_to_windows(windows, add_date_features, "minute")

> ### Feature 5: ***Separación del día en 4 partes***

In [13]:
def divide_time_day(auctions, installs, events, clicks):
    auctions["hour"] = auctions.date.dt.hour
    
    auctions.loc[(0 <= auctions["hour"]) & (auctions["hour"] < 6), 'timeofday'] = 0
    auctions.loc[(6 <= auctions["hour"]) & (auctions["hour"] < 13), 'timeofday'] = 1
    auctions.loc[(13 <= auctions["hour"]) & (auctions["hour"] < 19), 'timeofday'] = 2
    auctions.loc[(19 <= auctions["hour"]) & (auctions["hour"] < 24), 'timeofday'] = 3
    auctions["timeofday"] = auctions["timeofday"].astype(int)

    return auctions.set_index("ref_hash")

In [ ]:
windows = add_features_to_windows(windows, divide_time_day, "timeofday")

> ### Feature 6: ***Días feriados en Uruguay***

In [14]:
# En Uruguay hubo 3 días feriados en el rango de fecha que nos dieron
# El 18 y 19 de Abril fueron Jueves y Viernes Santo
# El 22 fue el día de Desembarco de los 33 orientales

def holidays_days(auctions, installs, events, clicks):
    auctions["day"] = auctions.date.dt.day
    auctions.loc[(22 == auctions["day"]) | (19 == auctions["day"]) | (18 == auctions["day"]), 'holidaysday'] = 1
    auctions.fillna(0)
    return auctions.set_index("ref_hash")

In [ ]:
windows = add_features_to_windows(windows, holidays_days, "holidaysday")

> ### Feature 7: ***Días lluviosos en Uruguay***

In [15]:
# Días que llovieron del 18 al 26: El 26 fue el único día que llovió
def raining_days(auctions, installs, events, clicks):
    auctions.loc[26 == auctions["day"], 'rainingdays'] = 1
    auctions.fillna(0)
    return auctions.set_index("ref_hash")

In [ ]:
windows = add_features_to_windows(windows, raining_days, "rainingdays")

> ### Feature 8: ***cantidad de clicks realizados por el usuario***

In [17]:
def clicks_by_user(auctions, installs, events, clicks):
    clicks['amount_of_clicks'] = 1
    inst_feature1 = clicks.groupby('ref_hash').agg({'amount_of_clicks': 'count'})
    return inst_feature1

In [ ]:
windows = add_features_to_windows(windows, clicks_by_user, "amount_of_clicks")

> ### Feature 9: ***¿realizó una instalación?***

In [ ]:
def user_has_installed(auctions, installs, events, clicks):
    installs['has_installed'] = 1
    inst_feature2 = installs.groupby('ref_hash').agg({'has_installed': 'first'})
    return inst_feature2

In [ ]:
windows = add_features_to_windows(windows, user_has_installed, "has_installed")

> ### Feature 10: ***¿realizó un click el último día?***

In [ ]:
def user_clicks_last_day(auctions, installs, events, clicks):
    inst_feature3 = clicks.loc[clicks["created"].dt.day == 20]
    inst_feature3['user_clicked_last_day'] = 1
    inst_feature3 = inst_feature3.groupby('ref_hash').agg({'user_clicked_last_day' : 'first'})
    return inst_feature3

In [ ]:
windows = add_features_to_windows(windows, user_clicks_last_day, "user_clicked_last_day")

> ### Feature 11: ***¿realizó una instalación el último día?***

In [18]:
def user_installs_last_day(auctions, installs, events, clicks):
    inst_feature4 = installs.loc[installs["created"].dt.day == 20]
    inst_feature4['user_installed_last_day'] = 1
    inst_feature4 = inst_feature4.groupby('ref_hash').agg({'user_installed_last_day' : 'first'})
    return inst_feature4

In [ ]:
windows = add_features_to_windows(windows, user_installs_last_day, "user_installed_last_day")

> ### Feature 12: ***cantidad de instalaciones realizadas***

In [19]:
def users_amount_installs(auctions, installs, events, clicks):
    installs['amount_of_installs'] = 1
    inst_feature5 = installs.groupby('ref_hash').agg({'amount_of_installs': 'count'})
    return inst_feature5

In [ ]:
windows = add_features_to_windows(windows, users_amount_installs, "amount_of_installs")

> ### Feature 13: ***Máximo de clicks en un día***

In [21]:
def max_clicks_day(auctions, installs, events, clicks):
    clicks["day"] = clicks_18_20.created.dt.day
    clicks_feature5 = clicks_18_20.groupby(["ref_hash", "day"]).agg({"cantidad":"sum"})
    clicks_feature5 = pd.DataFrame(clicks_feature5.reset_index().groupby('ref_hash')['cantidad'].max()).reset_index()
    clicks_feature5 = clicks_feature5.rename(columns={"cantidad":"cant_max_day"})
    return clicks_feature5.set_index("ref_hash")

In [ ]:
windows = add_features_to_windows(windows, min_clicks_day, "cant_max_day")

> ### Feature 14: ***Mínimo de clicks en un día***

In [ ]:
def min_clicks_day(auctions, installs, events, clicks):
    clicks["day"] = clicks_18_20.created.dt.day
    clicks_feature5 = clicks_18_20.groupby(["ref_hash", "day"]).agg({"cantidad":"sum"})
    clicks_feature5 = pd.DataFrame(clicks_feature5.reset_index().groupby('ref_hash')['cantidad'].min()).reset_index()
    clicks_feature5 = clicks_feature5.rename(columns={"cantidad":"cant_min_day"})
    return clicks_feature5.set_index("ref_hash")

In [ ]:
windows = add_features_to_windows(windows, min_clicks_day, "cant_min_day")

> ### Feature 15: ***Promedio de tiempo entre clicks***

In [ ]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name, function_type):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].fillna(259200) #to check
    df_sorted = df_sorted.groupby(feature_id_name).agg({'time_to_reappear': function_type})
    return df_sorted

In [23]:
def mean_time_click(auctions, installs, events, clicks):
    clicks_feature7 = calculate_time_to_reappear(clicks, "created", "ref_hash", "mean")
    clicks_feature7.rename(column={"time_to_reappear":"mean_time_click"}, inplace=True)
    return clicks_feature7

In [ ]:
windows = add_features_to_windows(windows, mean_time_click, "mean_time_click")

> ### Feature 16: ***Tiempo máximo entre clicks***

In [ ]:
def max_time_click(auctions, installs, events, clicks):
    clicks_feature7 = calculate_time_to_reappear(clicks, "created", "ref_hash", "max")
    clicks_feature7.rename(column={"time_to_reappear":"max_time_click"}, inplace=True)
    return clicks_feature7

In [ ]:
windows = add_features_to_windows(windows, max_time_click, "max_time_click")

> ### Feature 17: ***Tiempo mínimo entre clicks***

In [ ]:
def min_time_click(auctions, installs, events, clicks):
    clicks_feature7 = calculate_time_to_reappear(clicks, "created", "ref_hash", "min")
    clicks_feature7.rename(column={"time_to_reappear":"min_time_click"}, inplace=True)
    return clicks_feature7

In [ ]:
windows = add_features_to_windows(windows, min_time_click, "min_time_click")